In [85]:
import requests
from urllib import request
from bs4 import BeautifulSoup
import time
import pandas as pd

In [86]:
code=["008770","336260","009830",
      "086520","011930","018000",
      "035720","035420","036570",
      "018260","005930","000660",
      "011790","014680","000990",
      "023530","004170","028260",
      "026960","105560","071050","055550",
      "316140","086790"]
company=["호텔신라","두산퓨얼셀","한화솔루션",
      "에코프로","신성이엔지","유니슨",
      "카카오","네이버","엔씨소프트",
      "sds","삼성전자","SK하이닉스",
      "skc","한솔케미칼","DB하이텍",
      "롯데쇼핑","신세계","삼성물산",
      "동서","kb금융","한국금융지주","신한지주",
      "우리금융지주","하나금융지주"]

In [87]:
acode={"호텔신라":"008770","두산퓨얼셀":"336260","한화솔루션":"009830",
      "에코프로":"086520","신성이엔지":"011930","유니슨":"018000",
      "카카오":"035720","네이버":"035420","엔씨소프트":"036570",
      "sds":"018260","삼성전자":"005930","SK하이닉스":"000660",
      "skc":"011790","한솔케미칼":"014680","DB하이텍":"000990",
      "롯데쇼핑":"023530","신세계":"004170","삼성물산":"028260",
      "동서":"026960","kb금융":"105560","한국금융지주":"071050","신한지주":"055550",
      "우리금융지주":"316140","하나금융지주":"086790"}

In [94]:

BASE_URL = "https://gall.dcinside.com/board/lists?id=neostock&s_type=search_subject_memo&s_keyword=하나금융지주"
a = '/board/lists?id=neostock&s_type=search_subject_memo&s_keyword=하나금융지주'
headers={'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/93.0.4577.63 Safari/537.36'}
response = requests.get(BASE_URL,headers=headers)
soup = BeautifulSoup(response.text,'html.parser')
items = soup.find('div',{'class':"bottom_paging_box"})

#리스트 만들기 url
url_list=[a]
for i in items.find_all('a'):
    url_url = i['href']

    url_list.append(url_url)

In [95]:
from pymongo import MongoClient
client = MongoClient('localhost', 27017)
db = client['DC']
collection = db['DC_title_crawl']





page=30


        
def title_craw(ls):

    
    date_stop = "2021-08-19"
    seen_date = db.DC_title_crawl.find({"code":acode}).sort('date', -1).limit(1)
    
    seen_date = list(seen_date)
    if not seen_date:
        
        seen_date = "2021.08.19 00:00"

    for i in range(0,len(ls)-1):

        response = requests.get('https://gall.dcinside.com' + str(ls[i]),headers=headers)
        soup = BeautifulSoup(response.content,'html.parser')
        items = soup.find_all("table",{'class':'gall_list'})
        contents = soup.find('tbody').find_all('tr')

    for j in contents:

        if j.find('td',{'class':'gall_writer ub-writer'}).text=='운영자':
            pass
        else:

            #new_dict={}
            #제목
           # title = j.find('a').text
           # Title.append(title)


        #날짜
            date_tag = j.find('td',{'class':'gall_date'})

            date_dict = date_tag.attrs


            if date_dict['title'][:10] <= date_stop:
                TF = True
                return -1
            else:

               # Date.append(date_dict['title'])



                #제목
                title = j.find('a').text
               # Title.append(title)



                            #추천수
                recommend_tag = j.find('td', class_='gall_recommend')
                recommend = recommend_tag.text
                #Rec.append(recommend)

                            #조회수

                views_tag = j.find('td', class_='gall_count')
                views = views_tag.text
                #View.append(views)
                
                put_data = {
                                 'code':"086790",
                                 'date':date_dict['title'],
                                  'title':title,
                                  'Rec':recommend,
                                  'View':views           }

                db.DC_title_crawl.insert_one(put_data)
                
       




In [96]:
if __name__ == "__main__" :
    
  
    title_craw(url_list)

    k = 0


    while k< page :
        response = requests.get('https://gall.dcinside.com' + str(url_list[-1]),headers=headers)
        soup = BeautifulSoup(response.content,'html.parser')
        items = soup.find('div',{'class':"bottom_paging_box"})
        url_list=[str(url_list[-1])]

        for i in items.find_all('a'):
            url_url = i['href']
            url_list.append(url_url)
        url_list.pop(1) 


        if title_craw(url_list)==-1:
            break
        else:
            k=k+1








